# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [3]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [4]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [5]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [6]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [7]:
import glob, shutil

In [8]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [9]:
########################
# 이 셀부터 코드 작성하세요
########################
# 3. 데이터를 Training set | Validation set으로 분할하세요.

from sklearn.model_selection import train_test_split

list_all = []

for dir in glob.glob('/content/Dataset/*[0-9]*'):
  images = glob.glob(dir + '/*.jpg')
  jsons =  glob.glob(dir + '/*.json')
  images.sort()
  jsons.sort()
  for image, json in zip(images, jsons):
    list_all.append((image, json))

print(len(list_all))

trains, tests = train_test_split(list_all, test_size = 0.2, random_state = 2023, shuffle = True)

print(len(tests), len(trains))

5218
1044 4174


In [10]:
import pandas as pd

In [11]:
train_class = []
test_class = []
for i in trains:
  train_class.append(i[0][:i[0].find('_')].split('/')[4])
for i in tests:
  test_class.append(i[0][:i[0].find('_')].split('/')[4])

display(pd.DataFrame(train_class).value_counts())
print('-'*100)
display(pd.DataFrame(test_class).value_counts())

50000    699
1000     695
10000    694
5000     683
10       353
100      353
50       350
500      347
dtype: int64

----------------------------------------------------------------------------------------------------


5000     184
10000    173
50000    171
1000     163
500       93
50        90
100       87
10        83
dtype: int64

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [12]:
import os, json

In [13]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [14]:
len(trains)

4174

In [15]:
# won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
# label_dict = {}
# for idx, won in enumerate(won_list):
#   label_dict[won] = idx

# label_dict

In [16]:
s1 = set([])
for i in tests:
  with open(i[1], "r") as f:
	  tmp = json.load(f)

  for k in tmp['shapes']:
    s1.add(k['label'])

name_lt = []
for i in s1:
  name_lt.append(i[:i.rfind('_')])

name_set = set(name_lt)

label_dict = {}
for idx, won in enumerate(name_set):
  label_dict[won] = idx

label_dict

{'Thousand': 0,
 'Five_Thousand': 1,
 'Ten': 2,
 'Fifty': 3,
 'Ten_Thousand': 4,
 'Five_Hundred': 5,
 'Hundred': 6,
 'Fifty_Thousand': 7}

In [17]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################


train_images_path = '/content/Dataset/images/train'
test_images_path = '/content/Dataset/images/val'
train_labels_path = '/content/Dataset/labels/train'
test_labels_path = '/content/Dataset/labels/val'

for train in trains:

  if os.path.exists(train_images_path + '/' + train[0].split('/')[4]):
    pass
  else:
    shutil.copyfile(train[0], train_images_path + '/' + train[0].split('/')[4])

  train_file_name = train[0].split('/')[4].split('.')[0]

  with open(train[1], "r") as f:
	  train_json = json.load(f)

  with open(train_labels_path + '/' + train_file_name + '.txt', 'w') as f:
    image_w = train_json['imageWidth']
    image_h = train_json['imageHeight']
    for obj in train_json['shapes']:
      x1, y1 = obj['points'][0][0], obj['points'][0][1]
      x2, y2 = obj['points'][1][0], obj['points'][1][1]
      x_center = ((x1 + x2) / 2) / image_w
      y_center = ((y1 + y2) / 2) / image_h
      width = abs(x1-x2) / image_w
      height = abs(y1-y2) / image_h
      label = obj['label'][:obj['label'].rfind('_')]
      f.write(f'{label_dict[label]} {x_center} {y_center} {width} {height}\n')


for test in tests:

  if os.path.exists(test_images_path + '/' + test[0].split('/')[4]):
    pass
  else:
    shutil.copyfile(test[0], test_images_path + '/' + test[0].split('/')[4])

  test_file_name = test[0].split('/')[4].split('.')[0]

  with open(test[1], "r") as f:
	  test_json = json.load(f)

  with open(test_labels_path + '/' + test_file_name + '.txt', 'w') as f:
    image_w = test_json['imageWidth']
    image_h = test_json['imageHeight']
    for obj in test_json['shapes']:
      x1, y1 = obj['points'][0][0], obj['points'][0][1]
      x2, y2 = obj['points'][1][0], obj['points'][1][1]
      x_center = ((x1 + x2) / 2) / image_w
      y_center = ((y1 + y2) / 2) / image_h
      width = abs(x1-x2) / image_w
      height = abs(y1-y2) / image_h
      label = obj['label'][:obj['label'].rfind('_')]
      f.write(f'{label_dict[label]} {x_center} {y_center} {width} {height}\n')


In [18]:
print(len(os.listdir('/content/Dataset/images/val')), len(os.listdir('/content/Dataset/images/train')))
print(len(os.listdir('/content/Dataset/labels/val')), len(os.listdir('/content/Dataset/labels/train')))

1044 4174
1044 4174


### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [19]:
import yaml

In [20]:
label_dict_reverse = {}

for i,k in label_dict.items():
  label_dict_reverse[k]=i

label_dict_reverse

{0: 'Thousand',
 1: 'Five_Thousand',
 2: 'Ten',
 3: 'Fifty',
 4: 'Ten_Thousand',
 5: 'Five_Hundred',
 6: 'Hundred',
 7: 'Fifty_Thousand'}

In [21]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [22]:
yaml_data = {'path' : '/content/Dataset',
           'train' : '/content/Dataset/images/train',
           'val' : '/content/Dataset/images/val',
           'names': label_dict_reverse}

In [23]:
########################
# 이 셀부터 코드 작성하세요
########################

with open('/content/Dataset/money.yaml', 'w') as f :
  yaml.dump(yaml_data, f)


## 3.모델링

### (1) 모델 라이브러리 설치
---

In [24]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15994, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15994 (delta 18), reused 18 (delta 12), pack-reused 15967
Receiving objects: 100% (15994/15994), 14.64 MiB | 21.57 MiB/s, done.
Resolving deltas: 100% (10980/10980), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [27]:
# ########################
# # 이 셀부터 코드 작성하세요
# ########################
os.chdir('/content/yolov5')

# !python train.py --img 640 --epochs 10 --data /content/Dataset/money.yaml --weights /content/drive/MyDrive/dd/20230921mf/yolov5n.pt --cache \
#                  --project ./works --name work1 --exist-ok

In [28]:
!python train.py --img 640 --epochs 20 --data /content/Dataset/money.yaml --weights /content/drive/MyDrive/dd/20230921mf/yolov5n.pt --cache \
                 --project /content/drive/MyDrive/works --name work4 --exist-ok

train: weights=/content/drive/MyDrive/dd/20230921mf/yolov5n.pt, cfg=, data=/content/Dataset/money.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/works, name=work4, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [29]:
########################
# 이 셀부터 코드 작성하세요
########################
!python detect.py --source /content/drive/MyDrive/dd/20230921mf/KakaoTalk_20230925_105811384.mp4  --weights /content/drive/MyDrive/works/work4/weights/best.pt \
                 --project /content/drive/MyDrive/works --name work2_test --exist-ok

detect: weights=['/content/drive/MyDrive/works/work4/weights/best.pt'], source=/content/drive/MyDrive/dd/20230921mf/KakaoTalk_20230925_105811384.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/works, name=work2_test, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs
video 1/1 (1/262) /content/drive/MyDrive/dd/20230921mf/KakaoTalk_20230925_105811384.mp4: 384x640 1 Thousand, 46.3ms
video 1/1 (2/262) /content/drive/MyDrive/dd/20230921mf/KakaoTalk_20230925_105811384.mp4: 384x640 1 Thousand, 6.0ms
video 1

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
!